In [23]:
import numpy as np
import pandas as pd

In [24]:
pitching = pd.read_csv("../data/pitching_stats_2024.csv")
game_logs = pd.read_csv("../data/game_logs_2024.csv")
team_batting = pd.read_csv("../data/team_batting_stats_2024.csv", index_col=False)

In [25]:
pitching_feature_info = pd.read_csv("../data/pitching_stats_reference.csv", index_col=False)

In [27]:
### Function fetching the information of the pitching/batting stats abbreviations
def feature_info(feature):
    fullname = pitching_feature_info[pitching_feature_info['Abbreviation'] == feature]["Full Name"].iloc[0]
    desc = pitching_feature_info[pitching_feature_info['Abbreviation'] == feature]["Description"].iloc[0]
    print(feature + " : " + fullname + " , " + desc)
        
#Examples
feature_info('IBB')
feature_info('SV')

IBB : Intentional Walks , Intentional bases on balls
SV : Saves , Games successfully closed in a save situation


### Explanation of the Pitching Features

In [6]:
for col in pitching.columns:
    feature_info(col, type='pitching')

Name : Name , Name
Age : Age , Age
Team : Team , Team
W : Wins , Number of games where pitcher was credited with a win
L : Losses , Number of games where pitcher was credited with a loss
WAR : Wins Above Replacement , Overall value compared to a replacement-level player
ERA : Earned Run Average , Average earned runs per 9 innings
G : Games , Total games appeared in
GS : Games Started , Total games started by the pitcher
CG : Complete Games , Games pitched start to finish
ShO : Shutouts , Complete games with 0 runs allowed
SV : Saves , Games successfully closed in a save situation
IP : Innings Pitched , Total innings pitched
TBF : Total Batters Faced , Total plate appearances faced
H : Hits Allowed , Total hits allowed
R : Runs Allowed , All runs allowed
ER : Earned Runs , Runs charged to pitcher (excluding errors)
HR : Home Runs Allowed , Total home runs given up
BB : Walks , Bases on balls allowed
IBB : Intentional Walks , Intentional bases on balls
HBP : Hit By Pitch , Number of batt

In [8]:
#Build a dictionary that holds team name, id, and abbreviation
team_dict = {}

for _, row in game_logs[['home_name', 'home_id', 'home_name_abbr']].drop_duplicates().iterrows():
    team_dict[row['home_name_abbr']] = {
        'name' : row['home_name'],
        'id' : int(row['home_id'])
    }

{'name': 'Arizona Diamondbacks', 'id': 109}

In [ ]:
#Concatenating Batting Stats(by team) into the game_logs dataframe